In [91]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [92]:
df = pd.read_csv('data/train_data.csv')

In [93]:
# print(df.head())
df.describe()
# print(df[df['bedrooms'] == 0]) # 9 houses

# plt.scatter(df['bedrooms'], df['price'])
# plt.scatter(df['bathrooms'], df['price'])
# plt.scatter(df['sqft_living'], df['price']) # linear
# plt.scatter(df['floors'], df['price'])
# plt.scatter(df['waterfront'], df['price'])
# plt.scatter(df['view'], df['price'])
# plt.scatter(df['condition'], df['price']) # normal ish
# plt.scatter(df['grade'], df['price']) # linear ish
# plt.scatter(df['sqft_above'], df['price'])
# plt.scatter(df['yr_built'], df['price'])
# plt.scatter(df['yr_renovated'], df['price'])
# plt.scatter(df['sqft_living15'], df['price']) # linear ish
# plt.scatter(df['sqft_lot15'], df['price'])

,Unnamed: 0,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
count,17290.000000,1.729000e+04,17290.000000,17290.000000,17290.000000,1.729000e+04,17290.000000,17290.000000,17290.000000,17290.000000,...,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,17290.000000,1.729000e+04
mean,10821.135454,4.577035e+09,3.366455,2.111567,2074.586293,1.521099e+04,1.493522,0.007750,0.234182,3.410700,...,1782.821920,291.764372,1970.915095,84.596992,98078.057779,47.559674,-122.214147,1984.131463,12790.231984,5.378806e+05
std,6233.894051,2.877097e+09,0.934061,0.765353,903.771540,4.282989e+04,0.540177,0.087696,0.763897,0.651334,...,816.765091,441.081555,29.418996,402.087484,53.559522,0.138770,0.140738,681.613827,27728.922431,3.539798e+05
min,0.000000,1.000102e+06,0.000000,0.000000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,...,370.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,659.000000,7.500000e+04
25%,5408.500000,2.122785e+09,3.000000,1.500000,1420.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,...,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.470225,-122.329000,1490.000000,5100.000000,3.220000e+05
50%,10857.500000,3.904921e+09,3.000000,2.250000,1920.000000,7.605000e+03,1.500000,0.000000,0.000000,3.000000,...,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571300,-122.231000,1840.000000,7615.000000,4.500000e+05
75%,16222.500000,7.308600e+09,4.000000,2.500000,2550.000000,1.063075e+04,2.000000,0.000000,0.000000,4.000000,...,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.677700,-122.125000,2360.000000,10067.500000,6.450000e+05
max,21612.000000,9.900000e+09,33.000000,6.750000,9640.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,...,8020.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,871200.000000,5.570000e+06


In [97]:
features = pd.get_dummies(df)
features.iloc[:,5:].head(5)

##
labels = np.array(features['price'])
features = features.drop('price', axis=1)
feature_list = list(features.columns)

features = np.array(features)

train_features, test_features, train_labels, test_labels = train_test_split(df[[
        'bedrooms',
        'bathrooms',
        'sqft_living',
        'sqft_lot',
        'floors',
        'waterfront',
        'view',
        'condition',
        'grade',
        'sqft_above',
        'sqft_basement',
        'yr_built',
        'yr_renovated',
        'sqft_living15',
        'sqft_lot15',
    ]], df['price'], test_size = 0.25, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (12967, 15)
Training Labels Shape: (12967,)
Testing Features Shape: (4323, 15)
Testing Labels Shape: (4323,)


In [98]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('bedrooms')]
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

TypeError: '(slice(None, None, None), 2)' is an invalid key

In [99]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 50, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=50, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [100]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 116357.27


In [101]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 76.01 %.


In [13]:
x_train, x_test, y_train, y_test = train_test_split(
    df[[
        'bedrooms',
        'bathrooms',
        'sqft_living',
        'sqft_lot',
        'floors',
        'waterfront',
        'view',
        'condition',
        'grade',
        'sqft_above',
        'sqft_basement',
        'yr_built',
        'yr_renovated',
        'sqft_living15',
        'sqft_lot15',
    ]], 
    df['price'], 
    random_state=1
)

In [5]:
regr = linear_model.LinearRegression()
regr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:
clf = GridSearchCV(
    DecisionTreeClassifier(),
    {
        'max_depth' : [3, 5, 7, 9, None],
        'min_samples_leaf' : [1, 2, 3, 5],
        'max_features' : [4, 8, None],
    },
    cv=5
)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=5, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [3, 5, 7, 9, None],
                    

In [103]:
def rmse(predictions, actual):
    return np.sqrt(np.mean((predictions - actual)**2))

print('Linear Regression:', rmse(regr.predict(x_test), y_test))
print('Decision Tree', rmse(clf.predict(x_test), y_test))
print('Random Forest', rmse(rf.predict(test_features), test_labels))

Linear Regression: 215810.57967104393
Decision Tree 303037.8720319245
Random Forest 182905.83551883692


# Submission File Generation

In [8]:
dfTest = pd.read_csv('data/val_data.csv').rename({'Unnamed: 0': 'Unique_idx'}, axis='columns')

In [9]:
dfTest['price'] = regr.predict(dfTest[[
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'waterfront',
    'view',
    'condition',
    'grade',
    'sqft_above',
    'sqft_basement',
    'yr_built',
    'yr_renovated',
    'sqft_living15',
    'sqft_lot15',
]])

In [10]:
dfTest[['Unique_idx', 'price']].to_csv('submission.csv', index=None, header=True)